In [2]:
import sagemaker
from sklearn.model_selection import train_test_split
import boto3
import pandas as pd

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\shiro\AppData\Local\sagemaker\sagemaker\config.yaml


In [3]:
sm_boto3 = boto3.client('sagemaker')
sess = sagemaker.Session()
region = sess.boto_session.region_name
bucket = 'awssageprojectbucket'
print("Using Bucket " + bucket)

Using Bucket awssageprojectbucket


In [4]:
df = pd.read_csv('mob_price_classification_sage.csv')

In [5]:
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [6]:
df.shape

(2000, 21)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery_power  2000 non-null   int64  
 1   blue           2000 non-null   int64  
 2   clock_speed    2000 non-null   float64
 3   dual_sim       2000 non-null   int64  
 4   fc             2000 non-null   int64  
 5   four_g         2000 non-null   int64  
 6   int_memory     2000 non-null   int64  
 7   m_dep          2000 non-null   float64
 8   mobile_wt      2000 non-null   int64  
 9   n_cores        2000 non-null   int64  
 10  pc             2000 non-null   int64  
 11  px_height      2000 non-null   int64  
 12  px_width       2000 non-null   int64  
 13  ram            2000 non-null   int64  
 14  sc_h           2000 non-null   int64  
 15  sc_w           2000 non-null   int64  
 16  talk_time      2000 non-null   int64  
 17  three_g        2000 non-null   int64  
 18  touch_sc

In [8]:
df.describe()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
count,2000.000000,2000.0000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,...,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,1238.518500,0.4950,1.522250,0.509500,4.309500,0.521500,32.046500,0.501750,140.249000,4.520500,...,645.108000,1251.515500,2124.213000,12.306500,5.767000,11.011000,0.761500,0.503000,0.507000,1.500000
std,439.418206,0.5001,0.816004,0.500035,4.341444,0.499662,18.145715,0.288416,35.399655,2.287837,...,443.780811,432.199447,1084.732044,4.213245,4.356398,5.463955,0.426273,0.500116,0.500076,1.118314
min,501.000000,0.0000,0.500000,0.000000,0.000000,0.000000,2.000000,0.100000,80.000000,1.000000,...,0.000000,500.000000,256.000000,5.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000
25%,851.750000,0.0000,0.700000,0.000000,1.000000,0.000000,16.000000,0.200000,109.000000,3.000000,...,282.750000,874.750000,1207.500000,9.000000,2.000000,6.000000,1.000000,0.000000,0.000000,0.750000
50%,1226.000000,0.0000,1.500000,1.000000,3.000000,1.000000,32.000000,0.500000,141.000000,4.000000,...,564.000000,1247.000000,2146.500000,12.000000,5.000000,11.000000,1.000000,1.000000,1.000000,1.500000
75%,1615.250000,1.0000,2.200000,1.000000,7.000000,1.000000,48.000000,0.800000,170.000000,7.000000,...,947.250000,1633.000000,3064.500000,16.000000,9.000000,16.000000,1.000000,1.000000,1.000000,2.250000
max,1998.000000,1.0000,3.000000,1.000000,19.000000,1.000000,64.000000,1.000000,200.000000,8.000000,...,1960.000000,1998.000000,3998.000000,19.000000,18.000000,20.000000,1.000000,1.000000,1.000000,3.000000


In [9]:
df.columns

Index(['battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g',
       'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height',
       'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
       'touch_screen', 'wifi', 'price_range'],
      dtype='object')

In [10]:
df.isnull().mean()

battery_power    0.0
blue             0.0
clock_speed      0.0
dual_sim         0.0
fc               0.0
four_g           0.0
int_memory       0.0
m_dep            0.0
mobile_wt        0.0
n_cores          0.0
pc               0.0
px_height        0.0
px_width         0.0
ram              0.0
sc_h             0.0
sc_w             0.0
talk_time        0.0
three_g          0.0
touch_screen     0.0
wifi             0.0
price_range      0.0
dtype: float64

In [11]:
features = list(df.columns)
features

['battery_power',
 'blue',
 'clock_speed',
 'dual_sim',
 'fc',
 'four_g',
 'int_memory',
 'm_dep',
 'mobile_wt',
 'n_cores',
 'pc',
 'px_height',
 'px_width',
 'ram',
 'sc_h',
 'sc_w',
 'talk_time',
 'three_g',
 'touch_screen',
 'wifi',
 'price_range']

In [12]:
label = features.pop(-1)
label

'price_range'

In [13]:
x = df[features]
y = df[label]

In [14]:
x.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
0,842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0
2,563,1,0.5,1,2,1,41,0.9,145,5,6,1263,1716,2603,11,2,9,1,1,0
3,615,1,2.5,0,0,0,10,0.8,131,6,9,1216,1786,2769,16,8,11,1,0,0
4,1821,1,1.2,0,13,1,44,0.6,141,2,14,1208,1212,1411,8,2,15,1,1,0


In [15]:
y.head()

0    1
1    2
2    2
3    2
4    1
Name: price_range, dtype: int64

In [16]:
x.shape


(2000, 20)

In [17]:
y.value_counts()

price_range
1    500
2    500
3    500
0    500
Name: count, dtype: int64

In [18]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=0)

In [19]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1700, 20)
(300, 20)
(1700,)
(300,)


In [20]:
trainX = pd.DataFrame(X_train)
trainX[label] = y_train

testX = pd.DataFrame(X_test)
testX[label] = y_test

In [21]:
print(trainX.shape)
print(testX.shape)

(1700, 21)
(300, 21)


In [22]:
print(trainX.head())
print(testX.head())

      battery_power  blue  clock_speed  dual_sim  fc  four_g  int_memory  \
1452           1450     0          2.1         0   1       0          31   
1044           1218     1          2.8         1   3       0          39   
1279           1602     0          0.6         0  12       0          58   
674            1034     0          2.6         1   2       1          45   
1200            530     0          2.4         0   1       0          32   

      m_dep  mobile_wt  n_cores  ...  px_height  px_width   ram  sc_h  sc_w  \
1452    0.6        114        5  ...       1573      1639   794    11     5   
1044    0.8        150        7  ...       1122      1746  1667    10     0   
1279    0.4        170        1  ...       1259      1746  3622    17     2   
674     0.3        190        3  ...        182      1293   969    15     1   
1200    0.3         88        6  ...         48      1012   959    17     7   

      talk_time  three_g  touch_screen  wifi  price_range  
1452    

In [23]:
trainX.isnull().mean()

battery_power    0.0
blue             0.0
clock_speed      0.0
dual_sim         0.0
fc               0.0
four_g           0.0
int_memory       0.0
m_dep            0.0
mobile_wt        0.0
n_cores          0.0
pc               0.0
px_height        0.0
px_width         0.0
ram              0.0
sc_h             0.0
sc_w             0.0
talk_time        0.0
three_g          0.0
touch_screen     0.0
wifi             0.0
price_range      0.0
dtype: float64

In [24]:
testX.isnull().mean()

battery_power    0.0
blue             0.0
clock_speed      0.0
dual_sim         0.0
fc               0.0
four_g           0.0
int_memory       0.0
m_dep            0.0
mobile_wt        0.0
n_cores          0.0
pc               0.0
px_height        0.0
px_width         0.0
ram              0.0
sc_h             0.0
sc_w             0.0
talk_time        0.0
three_g          0.0
touch_screen     0.0
wifi             0.0
price_range      0.0
dtype: float64

In [25]:
trainX.to_csv('train-V-1.csv', index=False)
testX.to_csv('test-V-1.csv', index=False)

In [26]:
bucket

'awssageprojectbucket'

In [27]:
sk_prefix = "sagemaker/mobile_price_classification/sklearncontainer"

trainpath = sess.upload_data(
    path='S:/NEW/Projects/AWS_SageMaker_Project/train-V-1.csv',
    bucket=bucket,
    key_prefix=sk_prefix
)

testpath = sess.upload_data(
    path='S:/NEW/Projects/AWS_SageMaker_Project/test-V-1.csv',
    bucket=bucket,
    key_prefix=sk_prefix
)

print(trainpath)
print(testpath)

s3://awssageprojectbucket/sagemaker/mobile_price_classification/sklearncontainer/train-V-1.csv
s3://awssageprojectbucket/sagemaker/mobile_price_classification/sklearncontainer/test-V-1.csv


In [33]:
%%writefile script.py

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score
import sklearn
import joblib
import boto3
import pathlib
from io import StringIO
import argparse
import os
import numpy as np
import pandas as pd

def mode_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf

if __name__ == "__main__":
    print("[INFO] Extracting arguments")
    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script.
    parser.add_argument('--n_estimators', type=int, default=100)
    parser.add_argument('--random_state', type=int, default=0)

    # Data, model, and output directories
    parser.add_argument('--model-dir', type=str, default=os.environ.get('SM_MODEL_DIR'))
    parser.add_argument('--train', type=str, default=os.environ.get('SM_CHANNEL_TRAIN'))
    parser.add_argument('--test', type=str, default=os.environ.get('SM_CHANNEL_TEST'))
    parser.add_argument('--train-file', type=str, default="train-V-1.csv")
    parser.add_argument('--test-file', type=str, default="test-V-1.csv")

    args, _ = parser.parse_known_args()

    print("SKlearn Version: ", sklearn.__version__)
    print("Joblib Version: ", joblib.__version__)

    print("[INFO] Reading data")
    print()
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))

    features = list(train_df.columns)
    label = features.pop(-1)

    print("Building training and testing datasets")
    print()

    X_train = train_df[features]
    X_test = test_df[features]
    y_train = train_df[label]
    y_test = test_df[label]

    print('Column order: ')
    print(features)
    print()

    print("label column is: ", label)
    print()

    print("Data Shape")
    print()
    print("-----------Shape of training Data(85%)-----------")
    print(X_train.shape)
    print(y_train.shape)
    print()
    print("-----------Shape of testing Data(15%)-----------")
    print(X_test.shape)
    print(y_test.shape)
    print()

    print("Training of Model")
    print()
    model = RandomForestClassifier(
        n_estimators = args.n_estimators,
        random_state = args.random_state,
        verbose = 0 
    )
    model.fit(X_train, y_train)
    print()

    model_path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model, model_path)
    print("Model persisted at " + model_path)
    print()

    y_pred_test = model.predict(X_test)
    test_acc = accuracy_score(y_test, y_pred_test)
    test_rep = classification_report(y_test, y_pred_test)  

    print()
    print("-------Metrics Results for Testing Data-------------")
    print()
    print("Total Rows are: ", X_test.shape[0])
    print('[Testing] Model Accuracy: ', test_acc)
    print('[Testing] Testing Report: ')
    print(test_rep)







Overwriting script.py


In [34]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"

sklearn_estimator = SKLearn(
    entry_point = "script.py",
    role="arn:aws:iam::637423656000:role/Krish_Sagemaker_ProjectRole",   
    instance_count=1,
    instance_type = "ml.m5.large",
    framework_version = FRAMEWORK_VERSION,
    base_job_name = "RF-custom-sklearn",
    hyperparameters={
        "n_estimators": 100,
        "random_state": 0,
    },
    use_spot_instances = True,
    max_wait = 7200,
    max_run = 3600
)

In [35]:
sklearn_estimator.fit({"train": trainpath, "test": testpath}, wait=True)

INFO:sagemaker:Creating training-job with name: RF-custom-sklearn-2024-05-27-16-12-28-805


2024-05-27 16:12:30 Starting - Starting the training job...
2024-05-27 16:12:44 Starting - Preparing the instances for training...
2024-05-27 16:13:14 Downloading - Downloading input data...
2024-05-27 16:13:40 Downloading - Downloading the training image...
2024-05-27 16:14:30 Training - Training image download completed. Training in progress.
2024-05-27 16:14:30 Uploading - Uploading generated training model.2024-05-27 16:14:24,218 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-05-27 16:14:24,222 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-05-27 16:14:24,266 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-05-27 16:14:24,424 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-05-27 16:14:24,436 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-05-27 16:14:24,448 sagemaker-training-toolkit INF

In [36]:
sklearn_estimator.latest_training_job.wait(logs="None")
artifact = sm_boto3.describe_training_job(
    TrainingJobName= sklearn_estimator.latest_training_job.name
)["ModelArtifacts"]["S3ModelArtifacts"]

print("Model asrtifacts persisted at " + artifact)


2024-05-27 16:14:43 Starting - Preparing the instances for training
2024-05-27 16:14:43 Downloading - Downloading the training image
2024-05-27 16:14:43 Training - Training image download completed. Training in progress.
2024-05-27 16:14:43 Uploading - Uploading generated training model
2024-05-27 16:14:43 Completed - Training job completed
Model asrtifacts persisted at s3://sagemaker-us-east-1-637423656000/RF-custom-sklearn-2024-05-27-16-12-28-805/output/model.tar.gz


In [37]:
artifact

's3://sagemaker-us-east-1-637423656000/RF-custom-sklearn-2024-05-27-16-12-28-805/output/model.tar.gz'

In [44]:
from sagemaker.sklearn.model import SKLearnModel
from time import gmtime, strftime

model_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model = SKLearnModel(
    name = model_name,
    model_data= artifact,
    role = "arn:aws:iam::637423656000:role/Krish_Sagemaker_ProjectRole",
    entry_point="script.py",
    framework_version=FRAMEWORK_VERSION,
)



In [46]:
model_name

'Custom-sklearn-model-2024-05-27-18-39-08'

In [47]:
endpoint_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

predictor = model.deploy(
    initial_instance_count=1,
    instance_type = "ml.m4.xlarge",
    endpoint_name=endpoint_name
)

EndpointName=Custom-sklearn-model-2024-05-27-18-39-24


INFO:sagemaker:Creating model with name: Custom-sklearn-model-2024-05-27-18-39-08
INFO:sagemaker:Creating endpoint-config with name Custom-sklearn-model-2024-05-27-18-39-24
INFO:sagemaker:Creating endpoint with name Custom-sklearn-model-2024-05-27-18-39-24


-----!

In [48]:
endpoint_name

'Custom-sklearn-model-2024-05-27-18-39-24'

In [54]:
testX[features][0:10].values.tolist()

[[1454.0,
  1.0,
  0.5,
  1.0,
  1.0,
  0.0,
  34.0,
  0.7,
  83.0,
  4.0,
  3.0,
  250.0,
  1033.0,
  3419.0,
  7.0,
  5.0,
  5.0,
  1.0,
  1.0,
  0.0],
 [1092.0,
  1.0,
  0.5,
  1.0,
  10.0,
  0.0,
  11.0,
  0.5,
  167.0,
  3.0,
  14.0,
  468.0,
  571.0,
  737.0,
  14.0,
  4.0,
  11.0,
  0.0,
  1.0,
  0.0],
 [1524.0,
  1.0,
  1.8,
  1.0,
  0.0,
  0.0,
  10.0,
  0.6,
  174.0,
  4.0,
  1.0,
  154.0,
  550.0,
  2678.0,
  16.0,
  5.0,
  13.0,
  1.0,
  0.0,
  1.0],
 [1807.0,
  1.0,
  2.1,
  0.0,
  2.0,
  0.0,
  49.0,
  0.8,
  125.0,
  1.0,
  10.0,
  337.0,
  1384.0,
  1906.0,
  17.0,
  13.0,
  13.0,
  0.0,
  1.0,
  1.0],
 [1086.0,
  1.0,
  1.7,
  1.0,
  0.0,
  1.0,
  43.0,
  0.2,
  111.0,
  6.0,
  1.0,
  56.0,
  1150.0,
  3285.0,
  11.0,
  5.0,
  17.0,
  1.0,
  1.0,
  0.0],
 [909.0,
  1.0,
  0.5,
  1.0,
  9.0,
  0.0,
  30.0,
  0.4,
  97.0,
  3.0,
  10.0,
  290.0,
  773.0,
  594.0,
  12.0,
  0.0,
  4.0,
  1.0,
  1.0,
  1.0],
 [642.0,
  1.0,
  0.5,
  0.0,
  0.0,
  1.0,
  38.0,
  0.8,
  86.0

In [55]:
pred = predictor.predict(testX[features][0:10].values.tolist())

In [56]:
pred

array([3, 0, 2, 1, 3, 0, 0, 3, 3, 1], dtype=int64)

In [57]:
sm_boto3.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': '0c129eb5-4571-4420-a43d-77763720db2c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '0c129eb5-4571-4420-a43d-77763720db2c',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Mon, 27 May 2024 21:04:01 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}